Naive Bayes Algorithm

In [2]:
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv("play_tennis.csv")
data.drop(columns=["day"],inplace=True)
data.head()

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [9]:
data['play'].value_counts()

,count
play,
Yes,9
No,5


In [1]:
pyes = 9/14
pno = 5/14

In [2]:
print(pyes)
print(pno)

0.6428571428571429
0.35714285714285715


In [12]:
#outlook column
pd.crosstab(data['outlook'],data['play'])

play,No,Yes
outlook,,
Overcast,0,4
Rain,2,3
Sunny,3,2


In [3]:
povercastno = 0
praino = 2/5  #probability of rain no
psunnyno = 3/5

povercastyes = 4/9
prainyes = 3/9  #probability of rain yes
psunnyes = 2/9

In [15]:
#temp column
pd.crosstab(data['temp'],data['play'])

play,No,Yes
temp,,
Cool,1,3
Hot,2,2
Mild,2,4


In [4]:
pcooln =1/5
photn = 2/5
pmildn = 2/5

pcooly = 3/9
photy = 2/9
pmildy = 4/9

In [17]:
#humidity column
pd.crosstab(data['humidity'],data['play'])

play,No,Yes
humidity,,
High,4,3
Normal,1,6


In [5]:
phighno = 4/5
pnormno = 1/5

phighy = 3/9
pnormy = 6/9

In [19]:
#wind column
pd.crosstab(data['wind'],data['play'])

play,No,Yes
wind,,
Strong,3,3
Weak,2,6


In [6]:
pstrongN = 3/5
pweakNo = 2/5

pstrongY = 3/9
pweakY = 6/9

In [7]:
#outlook= sunny, temp=hot, humidity=high, wind=weak  match play r not?

PYes = pyes*psunnyes*photy*phighy*pweakY
print(PYes)

PNo = pno*psunnyno*photn*phighno*pweakNo
print(PNo)

0.007054673721340388
0.02742857142857143


In [4]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
class GaussianNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {cls: X[y == cls].mean(axis=0) for cls in self.classes}
        self.variance = {cls: X[y == cls].var(axis=0) for cls in self.classes}
        self.priors = {cls: len(X[y == cls]) / len(X) for cls in self.classes}

    def _gaussian_pdf(self, x, mean, variance):
        return (1 / np.sqrt(2 * np.pi * variance)) * np.exp(-((x - mean) ** 2) / (2 * variance))

    def predict(self, X):
        predictions = []
        for x in X:
            posteriors = {cls: np.log(self.priors[cls]) + np.sum(np.log(self._gaussian_pdf(x, self.mean[cls], self.variance[cls]))) for cls in self.classes}
            predictions.append(max(posteriors, key=posteriors.get))
        return np.array(predictions)

data = load_iris()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [3]:

model = GaussianNaiveBayes()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9777777777777777


Multinomial

In [5]:
class MultinomialNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.class_word_counts = {cls: X[y == cls].sum(axis=0) + 1 for cls in self.classes}  # Add 1 (Laplace smoothing)
        self.class_totals = {cls: self.class_word_counts[cls].sum() for cls in self.classes}
        self.priors = {cls: np.log(len(X[y == cls]) / len(X)) for cls in self.classes}

    def predict(self, X):
        predictions = []
        for x in X:
            posteriors = {cls: self.priors[cls] + np.sum(np.log(self.class_word_counts[cls] / self.class_totals[cls]) * x) for cls in self.classes}
            predictions.append(max(posteriors, key=posteriors.get))
        return np.array(predictions)

texts = ["buy cheap meds now", "win money instantly", "hello friend", "meeting at office", "cheap discount sale", "work project discussion"]
labels = np.array([1, 1, 0, 0, 1, 0])  # 1 = Spam, 0 = Not Spam

vectorizer = CountVectorizer(binary=False)  
X = vectorizer.fit_transform(texts).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

model = MultinomialNaiveBayes()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Multinomial Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred))


Multinomial Naïve Bayes Accuracy: 0.5


Bernoulli

In [6]:
class BernoulliNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.feature_prob = {cls: (X[y == cls].sum(axis=0) + 1) / (len(X[y == cls]) + 2) for cls in self.classes}  # Laplace smoothing
        self.priors = {cls: np.log(len(X[y == cls]) / len(X)) for cls in self.classes}

    def predict(self, X):
        predictions = []
        for x in X:
            posteriors = {cls: self.priors[cls] + np.sum(np.log(self.feature_prob[cls]) * x + np.log(1 - self.feature_prob[cls]) * (1 - x)) for cls in self.classes}
            predictions.append(max(posteriors, key=posteriors.get))
        return np.array(predictions)

# Sample text dataset (Spam vs. Not Spam)
texts = ["buy cheap meds now", "win money instantly", "hello friend", "meeting at office", "cheap discount sale", "work project discussion"]
labels = np.array([1, 1, 0, 0, 1, 0])  # 1 = Spam, 0 = Not Spam

vectorizer = CountVectorizer(binary=True)  # Convert text to binary features (word presence/absence)
X = vectorizer.fit_transform(texts).toarray()
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

model = BernoulliNaiveBayes()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Bernoulli Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred))

Bernoulli Naïve Bayes Accuracy: 0.5
